# Constraining the total capacity per bus and carrier

In this small example, we limit the nominal capacity of generators of the same production carrier at the same bus. 

Therefore, we introduce a column `nom_min_{carrier}` and `nom_max_{carrier}` in the `buses` dataframe. These are then used as lower and upper bounds of generators of the same carrier at the same bus. 


We start with importing a small example network.

In [2]:
import pypsa
from pathlib import Path

n = pypsa.Network(str(Path(pypsa.__file__).parent.parent / 'examples'/
                      'ac-dc-meshed'/ 'ac-dc-data'))

Importing PyPSA from older version of PyPSA than current version 0.17.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network ac-dc-data has buses, carriers, generators, global_constraints, lines, links, loads


Now add a second wind generator at bus 'Frankfurt' and limit the combined capacity. 

In [3]:
n.add('Generator', 'Frankfurt Wind 2', bus='Frankfurt', capital_cost = 120,
      carrier='wind', p_nom_extendable=True)

n.buses.loc[['Frankfurt', 'Manchester'], 'nom_min_wind'] = 2000
n.buses.loc[['Frankfurt'],'nom_max_wind'] = 2200

We are running the lopf and check whether the constraint is fulfilled. 

In [4]:
n.lopf(pyomo=False)

print(n.generators.p_nom_opt)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.25s
INFO:pypsa.linopf:Solve linear problem using Glpk solver
INFO:pypsa.linopf:Optimization successful. Objective value: -1.38e+07


Manchester Wind     2000.0000
Manchester Gas         0.0000
Norway Wind          895.3730
Norway Gas            91.0017
Frankfurt Wind       100.0000
Frankfurt Gas        884.0930
Frankfurt Wind 2    2100.0000
Name: p_nom_opt, dtype: float64


Looks good! The generators of carrier 'wind' at bus 'Frankfurt' are just the limit of 2200 MW.  